# Ajuste de un modelo lineal

Este ejemplo muestra el proceso de ajuste de un modelo lineal con un perceptrón simple y como función de activación la función lineal. Se utilizarán los métodos de aprendizaje automático `Batch Learning` y `Online Learning`. Los cuales se detallarán más adelante.

El perceptrón recibe $x$ y el sesgo $b$ como entrada y se utiliza una tasa de aprendizaje de $0,05$. Los datos corresponden a la función lineal $f(x)=2x-10$. Se quiere conseguir mediante el método del descenso del gradiente los dos parámetros $w1$ y $b1$.

Para la realización del modelo solo se utilizarán las librerías `statics` y `numpy`

In [6]:
import statistics as st
import numpy as np

Los datos de entrada son de la función lineal descrita y se utilizan 13 datos para entrenar el modelo.

In [9]:
# Datos de entrada
xList = [-5,  -4,  -3,  -2,  -1,   0,  1,  2,  3,  4, 5, 6, 7]

# Datos de salida (la función es 2*x - 10)
yList = [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4]

alpha = 0.05

A continuación se definen las funciones que utilizarán los métodos de aprendizaje:

- `linear` es la función de activación.
- `forward` es la fase de estimación que le aplica la función de activación a la entrada multiplicada por el parámetro y suma el sesgo.
- `backward` es la fase de ajuste del parámetro mediante el descenso del gradiente, que utiliza la tasa de aprendizaje `alpha` y el gradiente. El gradiente es la derivada de la función de error con respecto al parámetro, que en nuestro caso es la función de error cuadrático.
- `error` es la función que calcula el error cuadrático entre el valor estimado y el valor observado en los datos. Para este caso la función de error es: $\sum_{i=1}^{n}(\hat{y_{i}}-y_{i})^{^{2}}$

In [10]:
def linear(x):
    return x

In [11]:
def forward(x, w, b):
    yEst = linear(w*x + b)
    return yEst

In [12]:
def backward(w, dE_w):
    return w - (alpha*dE_w)

In [13]:
def error(yEst, y):
    return np.power(yEst-y, 2)

Más adelante se definen las funciones `batch_learning` y `online_learning`, estos dos métodos presentan variables en común. En particular la condición de parada para el proceso de ajuste iterativo en ambos es $10^{-20}$. Los valores iniciales de los parámetros $w1$ y $b1$ son arbitrarios.

In [8]:
#Condición de parada
stopAt = 1e-20

#Peso y sesgo incial arbitrarios
w1 = 0.46
b1 = 0.27

### Batch Learning
Este método de aprendizaje automático consiste en procesar los datos de entrada por lotes, es decir, se utilizan los mismos parámetros $w1$ y $b1$ para un lote de los datos de entrada y solo después de calcular cada uno de las valores estimados y sus errores, se ajustan los parámetros y se sigue con el siguiente lote, así sucesivamente, hasta cumplir con la condición de parada o superar el número de iteraciones.

La funcion que se definió para este método fue `batch_learning(x, y, n_batch, numIter)`, donde `x` representa los datos de entrada, `y` los datos de salida, `n_batch` el número de datos por lote y `numIter` el número de iteraciones.


Este es el código principal de la función, aquí se dividen los datos de entrada en lotes de acuerdo a la variable `n_batch`, luego está  el ciclo principal que se ejecuta dentro de la función a lo sumo `numIter` veces. Se define una variable `i` para saber cual lote de los datos tomar y cual lote de los valores observados tomar para calcular los diferentes errores y luego ajustar los parámetros.

El proceso consiste en el siguiente ciclo:

- Realizar el paso de estimación denominado `forward`.
- Calcular el error cuadrático entre el valor estimado en la fase previa y el valor observado mediante la función `error`.
- Calcular los gradientes que consiste en calcular la derivada de la función de error con respecto a `w1` y con respecto a `b1`.
- Guardar los gradientes. 
- Aplicar el paso de ajuste `backward`, con el promedio de los gradientes almacenados.

In [15]:
def batch_learning(x, y, n_batch, numIter):
    echo = False
    #Condición de parada
    stopAt = 1e-20
    
    #Peso y sesgo incial arbitrarios
    w1 = 0.46
    b1 = 0.27
    
    #Separando los datos en lotes
    xBatch = [x[i:i + n_batch] for i in range(0, len(x), n_batch)]
    yBatch = [y[i:i + n_batch] for i in range(0, len(y), n_batch)]
    
    for i in range(numIter):
        j = i%len(xBatch)
        
        w1_Derivatives = []
        b1_Derivatives = []
        errors = []
        for index in range(len(xBatch[j])):
            x = xBatch[j][index]
            y = yBatch[j][index]
            if echo:
                # Imprimir valores
                print('Iteración '+str(i+1)+str(' de ')+str(numIter))
                print('w1:'+str(w1)+', b1:'+str(b1))
            
            # Cálculo de y Estimado 
            yEst = forward(x, w1, b1)
            if echo:
                print('y:'+str(y)+', yEst:'+str(yEst))
    
            # Cálculo del error
            sse = error(yEst, y)
            if echo:
                print('error '+str(sse))
                
            # Cálculo de las derivadas parciales del Error con 
            # respecto a los parámetros
            dE_w1 = 2*(yEst-y)*x
            dE_b1 = 2*(yEst-y)
            
            if echo:
                print('d3_w1:'+str(dE_w1)+', dE_b1:'+str(dE_b1))
            
            #Guardar los gradientes
            w1_Derivatives.append(dE_w1)
            b1_Derivatives.append(dE_b1)
            errors.append(sse)
        
        #Promedio de los gradientes
        dE_w1Batch = st.mean(w1_Derivatives)
        dE_b1Batch = st.mean(b1_Derivatives)
        
        # Ajustando los parámetros w1 y b1
        w1 = backward(w1, dE_w1Batch)
        b1 = backward(b1, dE_b1Batch)
        if (st.mean(errors)<stopAt):
            return b1,w1
    return b1,w1

Para comprobar el funcionamiento de la función la invocamos con los datos de entrada, los datos de salida, el 
número de lotes igual a `2` y un número máximo de iteraciones igual a `500`

In [18]:
b1, w1 = batch_learning(xList, yList, 2,500)
print(b1,w1)

-9.999999999704968 1.9999999999142035


# Online Learning
Esta forma de aprendizaje automático a diferencia de `Batch Learning` no utiliza lotes de datos para ajustar los parámetros, simplemente procesa paso por paso cada dato de entrada, calcula el valor estimado y su error, y justo después ajusta los parámetros.

En en la función básicamente se sigue el mismo proceso que `batch_learning` solo que no se guardan los gradientes y no se utiliza el promedio en ningún momento.

El proceso consiste en el siguiente ciclo:

- Realizar el paso de estimación denominado `forward`.
- Calcular el error cuadrático entre el valor estimado en la fase previa y el valor observado mediante la función `error`.
- Calcular los gradientes que consiste en calcular la derivada de la función de error con respecto a `w1` y con respecto a `b1`.
- Aplicar el paso de ajuste `backward`, directamente con el gradiene previamente calculado.

In [17]:
def online_learning():
    echo = False
    #Condición de parada
    stopAt = 1e-20
    
    #Peso y sesgo incial arbitrarios
    w1 = 0.46
    b1 = 0.27
    
    numIter = 500
    sizeOfData = np.size(xList)
    
    for i in range(numIter):
        
        index = i%sizeOfData
        x = xList[index]
        y = yList[index]
        
        if echo:
            # Imprimir valores
            print('Iteración '+str(i+1)+str(' de ')+str(numIter))
            print('Datos: x'+str(index)+':'+str(x)+', w1:'+str(w1)+', b1:'+str(b1))
        
        # Cálculo de y Estimado
        yEst = forward(x, w1, b1)
        if echo:
            print('y:'+str(y)+', yEst:'+str(yEst))
        
        # Cálculo del Error
        sse = error(yEst, y)
        if echo:
            print('error '+str(sse))
        
        # Cálculo de las derivadas parciales
        dE_w1 = 2*(yEst-y)*x
        dE_b1 = 2*(yEst-y)
        if echo:
            print('d3_w1:'+str(dE_w1)+', dE_b1:'+str(dE_b1))
        
        # Ajustando los parámetros w1 y b1
        w1 = backward(w1, dE_w1)
        b1 = backward(b1, dE_b1)
        if echo:
            print('After adjusting')
            print('w1:'+str(w1)+', b1:'+str(b1))
            print('')
        
        # Chequear condición de parada
        if (sse<stopAt):
            return b1, w1
    return b1, w1

In [18]:
b1, w1 = online_learning()
print(b1,w1)

-9.999999997782723 1.9999999995335829
